# アドレスの生成
## 可読性を高める工夫
予測されない秘密鍵を作成し、それを元に逆算できない公開鍵を生成したことで、ある程度の秘匿性を確保することができた。  
しかし、公開鍵は文字列が長く、可読性が低いため、エンコードを行なって誤読を防ぐという工夫がされる。  
具体的には、人が読みやすい形にするため、ハッシュ関数として RIPEMD-160 を利用して文字量を減らしたり、Base56Check エンコードを行ったりされる。  
## Base58
Base58とは、人が読み間違い安い文字列を排除したエンコード方式であり、アドレスを入力したり、書き取ったりする際に間違えにくいようにする手法。  
具体的には、数字の０やアルファベットのO,　小文字のlと大文字のI, プラス＋やスラッシュ/などが使われないようにする。
## Base58Check
公開鍵からアドレスを生成する際は、チェックサムを利用したBase58Checkが利用される。  
チェックサムはデータが間違っていないかどうかを確認するための手法で、元のデータから算出できるデータの一部を付加して利用する。  
これはBase58Checkエンコードの際に、チェックサムを読み込むことで記入ミスを発見できるようにするもの。  
ここでのチェックサムは、公開鍵のハッシュ値とその先頭にバージョンバイトをつなげたものからSHA-256で２回ハッシュ化したものの先頭4バイトを利用する。
なので、バージョンバイト＋公開鍵のハッシュ値＋チェックサム をBase58エンコードすることによってアドレスは作られる。
## アドレスを生成する
公開鍵からアドレスを作るプロセス
1. 公開鍵をSHA-256でハッシュ値①にハッシュ化
2. ハッシュ値①を RIPEMD-160 でハッシュ値② にハッシュ化
3. ハッシュ値②を Base58Checkエンコード

In [1]:
import os
import ecdsa
import hashlib
import base58

private_key = os.urandom(32)
public_key = ecdsa.SigningKey.from_string(private_key, curve=ecdsa.SECP256k1).verifying_key.to_string()

# 非圧縮公開鍵のプレフィックスの04を公開鍵に付加
prefix_and_pubkey = b"\x04" + public_key

# ハッシュ160を生成
intermediate = hashlib.sha256(prefix_and_pubkey).digest()
ripemd160 = hashlib.new('ripemd160')
ripemd160.update(intermediate)
hash160 = ripemd160.digest()

# 公開鍵のハッシュのバージョンプレフィックスである00と公開鍵八種を合体。
prefix_and_hash160 = b"\x00" + hash160

#hashlib.sha256が入れ子になっていることを確認！
double_hash = hashlib.sha256(hashlib.sha256(prefix_and_hash160).digest()).digest()
# 先頭の４バイトを取り出す。
checksum = double_hash[:4]
# チェックサムを取り出し、Base58エンコードすることでアドレスを生成
pre_address = prefix_and_hash160 + checksum

address = base58.b58encode(pre_address)
print(address.decode())


1FZKPNYSiGQYWXi8souRRrKNiU814NdNDg
